In [1]:
# 1 加载必要的库
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
# 2 定义超参数

# 每批处理的数据
BATCH_SIZE = 16 
# 训练cpu还是gpu
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# 训练的轮次
EPOCHS = 10

In [3]:
# 3 构建pipeline，对图像做处理
pipeline = transforms.Compose([
     # 将图片转换为tensor
     transforms.ToTensor(),
     # 正则化：降低模型复杂度
     transforms.Normalize((0.1307,), (0.3081))
 ])

In [4]:
# 4 下载、加载数据
from torch.utils.data import DataLoader

# 下载数据集
train_set = datasets.MNIST("data", train=True, download=True, transform=pipeline)

test_set = datasets.MNIST("data", train=False, download=True, transform=pipeline)

# 加载数据
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)

test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True)

In [5]:
# 5 构建网络模型
class Digit(nn.Module):
    def __init__(self):
        super().__init__()
        # 灰度图片的通道， 10：输出通道，5：kernel
        self.conv1 = nn.Conv2d(1, 10, 5)
        # 10：输出通道，20：输出通道，3：kernel
        self.conv2 = nn.Conv2d(10, 20, 3)
        
        # 20*10*10：输入通道，500：输出通道
        self.fc1 = nn.Linear(20*10*10, 500)
        # 500：输入通道，10：输出通道
        self.fc2 = nn.Linear(500, 10)
        
    def forward(self, x):
        # batch_size
        input_size = x.size(0)
        # 输入：batch*1*28*28，输出：batch*10*24*24 (28 - 5 + 1 = 24)
        x = self.conv1(x)
        # 保持shape不变，输出：batch*10*24*24 (激活函数)
        x = F.relu(x)
        # 输入：batch*10*24*24 输出：batch*10*12*12 (减半)
        x = F.max_pool2d(x, 2, 2)
        
        # 输入：batch*10*12*12 输出：batch*20*10*10
        x = self.conv2(x)
        x = F.relu(x)
        
        # 拉平，-1 自动计算维度，20*10*10 = 2000
        x = x.view(input_size, -1)
        
        # 输入：batch*2000 输出：batch*500
        x = self.fc1(x)
        # 保持shape不变
        x = F.relu(x)
        
        # 输入：batch*500 输出：batch*10
        x = self.fc2(x)
        
        # 计算分类后，每个数字的概率值
        output = F.log_softmax(x, dim=1)
        
        return output

In [6]:
# 6 定义优化器
model = Digit().to(DEVICE)

optimizer = optim.Adam(model.parameters())

In [7]:
# 7 定义训练方法
def train_model(model, device, train_loader, optimizer, epoch):
    # 模型训练
    model.train()
    for batch_index, (data, target) in enumerate(train_loader):
        # 部署到DEVICE上去
        data, target = data.to(device), target.to(device)
        # 梯度初始化为0
        optimizer.zero_grad()
        # 训练后的结果
        output = model(data)
        # 计算损失
        loss = F.cross_entropy(output, target)
        # 找到概率值最大的下标
        pred = output.max(1, keepdim=True)
        # pred = output.argmax(dim=1)
        # 反向传播
        loss.backward()
        # 参数优化
        optimizer.step()
        if batch_index % 3000 == 0:
            print("Train Epoch : {} \t Loss : {:.6f}".format(epoch, loss.item()))

In [8]:
# 8 定义测试方法
def test_model(model, device, test_loader):
    # 模型验证
    model.eval()
    # 正确率
    correct = 0.0
    # 测试损失
    test_loss = 0.0
    # 不会计算梯度，也不会进行反向传播
    with torch.no_grad():
        for data, target in test_loader:
            # 部署到device上
            data, target = data.to(device), target.to(device)
            # 测试数据
            output = model(data)
            # 计算测试损失
            test_loss += F.cross_entropy(output, target).item()
            # 找到概率值最大的下标
            pred = output.max(1, keepdim=True)[1] # 值， 索引
            # pred = torch.max(output, dim=1)
            # pred = output.argmax(dim=1)
            # 累计正确的值
            correct += pred.eq(target.view_as(pred)).sum().item()
            
        test_loss /= len(test_loader.dataset)
        print("Test Average loss : {:.4f}, Accuray : {:.3f}\n".format(
                test_loss, 100.0 * correct / len(test_loader.dataset))) 

In [9]:
# 9 调用 方法 7 & 8
for epoch in range(1, EPOCHS + 1):
    train_model(model, DEVICE, train_loader, optimizer, epoch)
    test_model(model, DEVICE, test_loader)

Train Epoch : 1 	 Loss : 2.308449
Train Epoch : 1 	 Loss : 0.012582
Test Average loss : 0.0031, Accuray : 98.390

Train Epoch : 2 	 Loss : 0.015619
Train Epoch : 2 	 Loss : 0.092979
Test Average loss : 0.0026, Accuray : 98.630

Train Epoch : 3 	 Loss : 0.223918
Train Epoch : 3 	 Loss : 0.001498
Test Average loss : 0.0026, Accuray : 98.630

Train Epoch : 4 	 Loss : 0.001047
Train Epoch : 4 	 Loss : 0.000053
Test Average loss : 0.0024, Accuray : 98.890

Train Epoch : 5 	 Loss : 0.000032
Train Epoch : 5 	 Loss : 0.006103
Test Average loss : 0.0030, Accuray : 98.820

Train Epoch : 6 	 Loss : 0.000098
Train Epoch : 6 	 Loss : 0.000013
Test Average loss : 0.0026, Accuray : 99.000

Train Epoch : 7 	 Loss : 0.000124
Train Epoch : 7 	 Loss : 0.001418
Test Average loss : 0.0028, Accuray : 98.970

Train Epoch : 8 	 Loss : 0.000003
Train Epoch : 8 	 Loss : 0.001643
Test Average loss : 0.0038, Accuray : 98.870

Train Epoch : 9 	 Loss : 0.000003
Train Epoch : 9 	 Loss : 0.000002
Test Average loss : 